In [ ]:
# ==============================================================================
# 1. SETUP, DEPENDENCIES & GPU CHECK
# ==============================================================================
import os
import subprocess
import sys
import shutil
import time
from datetime import datetime

# Install dependencies
print("📦 Installing Dependencies...")
pkgs = ["torchcrepe", "librosa", "torchaudio", "tqdm"]
for p in pkgs:
    subprocess.check_call([sys.executable, "-m", "pip", "install", p])

import torch
import torchcrepe
import torchaudio
import numpy as np
import glob
from tqdm import tqdm
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# --- STRICT CUDA CHECK ---
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🎮 HARDWARE STATUS: {device.upper()}")
if device == 'cpu':
    raise RuntimeError("❌ STOP! You are running on CPU. Go to Runtime > Change runtime type > Select T4 GPU.")
else:
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")

# ==============================================================================
# 2. MODEL RENAMING (Based on your Accuracy Table)
# ==============================================================================
MODEL_DIR = "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/testing_models"

# Mapping: 'best (X).pth' -> 'New_Name.pth'
mapping = {
    "best (15).pth": "OldCNN_NoSmooth.pth",
    "best (16).pth": "OldCNN_Smooth.pth",
    "best (17).pth": "OldCNN_Smooth_V2.pth",
    "best (21).pth": "DeepCNN_NoSmooth.pth",
    "best (18).pth": "DeepCNN_Smooth.pth",
    "best (20).pth": "CRNN_NoSmooth.pth",
    "best (19).pth": "CRNN_Smooth.pth"
}

print("\n🔄 RENAMING MODELS...")
if os.path.exists(MODEL_DIR):
    for old_name, new_name in mapping.items():
        old_path = os.path.join(MODEL_DIR, old_name)
        new_path = os.path.join(MODEL_DIR, new_name)

        if os.path.exists(old_path):
            os.rename(old_path, new_path)
            print(f"✅ Renamed: {old_name} -> {new_name}")
        elif os.path.exists(new_path):
            print(f"ℹ️ Already Renamed: {new_name}")
        else:
            print(f"⚠️ Not Found: {old_name}")
else:
    print(f"❌ Error: Directory {MODEL_DIR} not found.")

# ==============================================================================
# 3. MEGA DATA EXTRACTION (SMART UNZIP)
# ==============================================================================
# Define paths
ZIP_EVAL = "/content/drive/MyDrive/FINE_TUNE_V3/eval_originals_300.zip"
ZIP_TRAIN = "/content/drive/MyDrive/FINE_TUNE_V3/train_originals_1300.zip"

# Local temporary directories (Fast I/O)
AUDIO_DIR = "/content/all_audio"
PITCH_DIR = "/content/all_pitch"

# --- SAFETY WIPE FOR PITCH ONLY ---
# We wipe pitch to ensure fresh extraction, but check audio to save time
print(f"\n🧹 Clearing old pitch directory...")
if os.path.exists(PITCH_DIR): shutil.rmtree(PITCH_DIR)
os.makedirs(PITCH_DIR, exist_ok=True)

# --- SMART UNZIP LOGIC ---
# Only unzip if AUDIO_DIR is empty or doesn't exist
if not os.path.exists(AUDIO_DIR) or len(os.listdir(AUDIO_DIR)) < 100:
    print(f"📂 Audio directory empty/missing. Unzipping FRESH data...")
    if os.path.exists(AUDIO_DIR): shutil.rmtree(AUDIO_DIR)
    os.makedirs(AUDIO_DIR, exist_ok=True)

    print(f"   extracting Eval Set...")
    subprocess.run(f"unzip -q -n '{ZIP_EVAL}' -d '{AUDIO_DIR}'", shell=True)

    print(f"   extracting Train Set...")
    subprocess.run(f"unzip -q -n '{ZIP_TRAIN}' -d '{AUDIO_DIR}'", shell=True)
else:
    print(f"⏩ Audio directory already populated ({len(os.listdir(AUDIO_DIR))} files). Skipping Unzip.")

# Check total files
files = sorted(glob.glob(os.path.join(AUDIO_DIR, "*.wav")))
print(f"✅ Total Audio Files Ready: {len(files)}")

# ==============================================================================
# 4. GPU PITCH EXTRACTION (OPTIMIZED: ARGMAX + BATCHING)
# ==============================================================================
print(f"\n🎵 Starting CREPE Pitch Extraction on {len(files)} files...")
print("   (Using GPU Resampling + Argmax Decoding for Speed)")

# OPTIMIZATION SETTINGS
BATCH_SIZE = 2048
HOP_LENGTH = 160  # Must match training
DECODER = torchcrepe.decode.argmax # 3x Faster than Viterbi

for path in tqdm(files):
    name = os.path.basename(path).replace(".wav", ".npy")
    save_path = os.path.join(PITCH_DIR, name)

    if os.path.exists(save_path): continue

    try:
        # 1. Load Audio (CPU)
        audio, sr = torchaudio.load(path)

        # 2. Move to GPU IMMEDIATELY (Fast Resampling)
        audio = audio.to(device)

        # 3. Resample on GPU if needed
        if sr != 16000:
            audio = torchaudio.functional.resample(audio, sr, 16000)

        # 4. Mix to Mono on GPU
        if audio.shape[0] > 1:
            audio = audio.mean(dim=0, keepdim=True)

        # 5. CREPE Predict (GPU)
        pitch = torchcrepe.predict(
            audio,
            sample_rate=16000,
            hop_length=HOP_LENGTH,
            fmin=50,
            fmax=2000,
            model='tiny',
            decoder=DECODER,       # <--- SPEED OPTIMIZATION
            batch_size=BATCH_SIZE,
            device=device
        )

        # 6. Post-Process (CPU)
        pitch = pitch.squeeze().cpu().numpy()
        pitch = np.log1p(pitch) # Log scale matching training

        # 7. Save
        np.save(save_path, pitch.astype(np.float32))

    except Exception as e:
        print(f"❌ Error on {name}: {e}")

print(f"\n✅ SUCCESS: All {len(files)} pitch vectors saved to {PITCH_DIR}")

# ==============================================================================
# 5. SAVE PROCESSED DATA TO DRIVE (AUTO-ZIP)
# ==============================================================================
# 1. Config
SOURCE_DIR = "/content/all_pitch"  # Where your .npy files are right now
DRIVE_DEST_DIR = "/content/drive/MyDrive/FIND_TUNE" # Check if this path exists or needs creation
os.makedirs(DRIVE_DEST_DIR, exist_ok=True)

TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M")
ZIP_NAME = f"processed_pitch_vectors_{TIMESTAMP}.zip"
ZIP_PATH_LOCAL = f"/content/{ZIP_NAME}"
ZIP_PATH_DRIVE = os.path.join(DRIVE_DEST_DIR, ZIP_NAME)

print(f"\n📦 Zipping {len(os.listdir(SOURCE_DIR))} files from {SOURCE_DIR}...")

# 2. Create Zip (shutil.make_archive adds .zip automatically, so we strip it)
shutil.make_archive(ZIP_PATH_LOCAL.replace('.zip', ''), 'zip', SOURCE_DIR)

# 3. Copy to Drive
print(f"🚀 Uploading to Drive: {ZIP_PATH_DRIVE} ...")
shutil.copy(ZIP_PATH_LOCAL, ZIP_PATH_DRIVE)

if os.path.exists(ZIP_PATH_DRIVE):
    print(f"✅ SUCCESS! Data saved to: {ZIP_PATH_DRIVE}")
    print(f"   (File size: {os.path.getsize(ZIP_PATH_DRIVE) / 1024 / 1024:.2f} MB)")
else:
    print(f"❌ Error: File did not appear on Drive.")

📦 Installing Dependencies...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🎮 HARDWARE STATUS: CUDA
✅ GPU Detected: Tesla T4

🔄 RENAMING MODELS...
ℹ️ Already Renamed: OldCNN_NoSmooth.pth
ℹ️ Already Renamed: OldCNN_Smooth.pth
ℹ️ Already Renamed: OldCNN_Smooth_V2.pth
ℹ️ Already Renamed: DeepCNN_NoSmooth.pth
ℹ️ Already Renamed: DeepCNN_Smooth.pth
ℹ️ Already Renamed: CRNN_NoSmooth.pth
ℹ️ Already Renamed: CRNN_Smooth.pth

🧹 Clearing old pitch directory...
⏩ Audio directory already populated (1616 files). Skipping Unzip.
✅ Total Audio Files Ready: 1616

🎵 Starting CREPE Pitch Extraction on 1616 files...
   (Using GPU Resampling + Argmax Decoding for Speed)


100%|██████████| 1616/1616 [41:57<00:00,  1.56s/it]



✅ SUCCESS: All 1616 pitch vectors saved to /content/all_pitch

📦 Zipping 1616 files from /content/all_pitch...
🚀 Uploading to Drive: /content/drive/MyDrive/FIND_TUNE/processed_pitch_vectors_20260212_1454.zip ...
✅ SUCCESS! Data saved to: /content/drive/MyDrive/FIND_TUNE/processed_pitch_vectors_20260212_1454.zip
   (File size: 150.28 MB)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. OLD ARCHITECTURE (3-Layer CNN)
class OldCNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, 5, padding=2), nn.BatchNorm1d(32), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(32, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(
            nn.Linear(128, 128), nn.ReLU(), nn.Linear(128, embed_dim)
        )
    def forward_one(self, x):
        x = self.cnn(x).squeeze(-1)
        return F.normalize(self.fc(x), p=2, dim=1)

# 2. DEEP ARCHITECTURE (4-Layer CNN)
class DeepCNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, 5, padding=2), nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(
            nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, embed_dim)
        )
    def forward_one(self, x):
        x = self.cnn(x).squeeze(-1)
        return F.normalize(self.fc(x), p=2, dim=1)

# 3. CRNN ARCHITECTURE (CNN + LSTM)
class CRNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
        )
        self.lstm = nn.LSTM(256, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, embed_dim)
        )
    def forward_one(self, x):
        x = self.cnn(x)
        x = x.permute(0, 2, 1) # (Batch, Seq, Feat)
        self.lstm.flatten_parameters()
        out, _ = self.lstm(x)
        out = torch.mean(out, dim=1)
        return F.normalize(self.fc(out), p=2, dim=1)

In [ ]:
import scipy.signal as sg
import random
from collections import defaultdict

# --- UTILS ---
def smooth_pitch(pitch):
    return sg.medfilt(pitch, kernel_size=5).astype(np.float32)

def augment_clip(arr, mode='clean'):
    arr = arr.copy()

    if mode == 'clean':
        return arr

    elif mode == 'soft':
        # Light noise + light vibrato
        arr += np.random.normal(0, 0.02, size=len(arr))
        return arr

    elif mode == 'hard':
        # Noise + Key Shift + Time Warp
        arr += np.random.normal(0, 0.06, size=len(arr)) # Jitter
        semitones = np.random.uniform(-3, 3)
        arr[arr > 0] += semitones * 0.057 # Shift
        if random.random() < 0.8: # Warp
            rate = np.random.uniform(0.85, 1.15)
            old_idx = np.arange(len(arr))
            new_len = int(len(arr) * rate)
            new_idx = np.linspace(0, len(arr)-1, new_len)
            arr = np.interp(new_idx, old_idx, arr)
            # Force length
            if len(arr) < 300: arr = np.pad(arr, (0, 300 - len(arr)), 'constant')
            else: arr = arr[:300]
        return arr

    elif mode == 'ultra':
        # "Limit Testing": Heavy Noise + Extreme Drift + Dropout
        arr += np.random.normal(0, 0.1, size=len(arr)) # Heavy Jitter

        # Extreme Tempo Drift (Slow down then speed up)
        rate = np.linspace(0.8, 1.2, len(arr))
        old_idx = np.arange(len(arr))
        # Cumulative sum gives the new index mapping
        new_idx = np.cumsum(rate)
        new_idx = new_idx / new_idx[-1] * (len(arr)-1)
        arr = np.interp(np.arange(len(arr)), new_idx, arr)

        # Signal Dropout (Simulate bad mic)
        mask_size = random.randint(10, 50)
        start = random.randint(0, len(arr)-mask_size)
        arr[start:start+mask_size] = 0

        return arr

    return arr

def build_db_for_model(model, pitch_dir, smooth_db=False):
    files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))
    db_embeds = []
    metadata = []

    model.eval()
    with torch.no_grad():
        for f in tqdm(files, desc="Building DB", leave=False):
            arr = np.load(f)
            if smooth_db: arr = smooth_pitch(arr)

            # Windowing
            windows = []
            offsets = []
            i = 0
            while i + 300 <= len(arr):
                crop = arr[i:i+300]
                if np.mean(crop > 0) > 0.1: # Skip silence
                    windows.append(crop)
                    offsets.append(i/100.0)
                i += 150

            if not windows: continue

            # Batch Inference
            w_tensor = torch.from_numpy(np.stack(windows)).float().unsqueeze(1).to(DEVICE)
            embeddings = model.forward_one(w_tensor)

            db_embeds.append(embeddings)
            song_name = os.path.basename(f).replace(".npy","")
            for t in offsets:
                metadata.append((song_name, t))

    if not db_embeds: return None, None
    return torch.cat(db_embeds), metadata

def run_evaluation(model, db_tensor, db_meta, pitch_dir, smooth_query=False):
    files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))

    modes = ['Clean', 'Soft', 'Hard', 'Ultra']
    results = {m: {'top1':0, 'top5':0, 'top10':0} for m in modes}
    total_clips = 0

    model.eval()

    # Eval Loop: All songs, 3 random clips each
    for f in tqdm(files, desc="Eval Loop"):
        full_arr = np.load(f)
        if len(full_arr) < 500: continue
        target_name = os.path.basename(f).replace(".npy","")

        # Take 3 random starting points
        starts = np.random.randint(0, len(full_arr)-300, 3)

        for s in starts:
            base_clip = full_arr[s:s+300]
            if smooth_query: base_clip = smooth_pitch(base_clip)

            total_clips += 1

            for mode in modes:
                # Augment
                aug_clip = augment_clip(base_clip, mode=mode.lower())

                # Embed Query
                q_tensor = torch.from_numpy(aug_clip).float().unsqueeze(0).unsqueeze(0).to(DEVICE)
                with torch.no_grad():
                    q_emb = model.forward_one(q_tensor)

                # Geometric Scoring
                dists = torch.cdist(q_emb, db_tensor) # (1, DB_Size)
                # Optimization: Only top 50 candidates for voting
                vals, inds = torch.topk(dists, k=50, largest=False)

                votes = defaultdict(float)
                q_time = 0 # Single clip query

                for k in range(50):
                    idx = inds[0, k].item()
                    dist = vals[0, k].item()
                    m_name, m_time = db_meta[idx]

                    # Alignment check (Assuming user query is start of a segment)
                    # For single clip query, we trust distance mostly
                    score = 1.0 / (dist + 1e-4)
                    votes[m_name] += score

                ranked = sorted(votes.items(), key=lambda x: x[1], reverse=True)
                top_names = [x[0] for x in ranked]

                if not top_names: continue

                if top_names[0] == target_name: results[mode]['top1'] += 1
                if target_name in top_names[:5]: results[mode]['top5'] += 1
                if target_name in top_names[:10]: results[mode]['top10'] += 1

    return results, total_clips

In [ ]:
# ==============================================================================
# 4. MASTER EVALUATION LOOP
# ==============================================================================
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define the Experiment List based on your renaming
# Format: (Filename, Class, Needs_Smoothing)
experiments = [
    ("OldCNN_NoSmooth.pth", OldCNN, False),
    ("OldCNN_Smooth.pth",   OldCNN, True),
    ("DeepCNN_NoSmooth.pth", DeepCNN, False),
    ("DeepCNN_Smooth.pth",   DeepCNN, True),
    ("CRNN_NoSmooth.pth",    CRNN, False),
    ("CRNN_Smooth.pth",      CRNN, True),
]

print(f"🚀 STARTING MULTI-MODEL EVALUATION")
print(f"📂 Pitch Dir: {PITCH_DIR}")
print(f"🎮 Device: {DEVICE}")

final_report = {}

for model_name, ModelClass, smooth_on in experiments:
    model_path = os.path.join(MODEL_DIR, model_name)

    if not os.path.exists(model_path):
        print(f"\n⚠️ SKIPPING {model_name} (File not found)")
        continue

    print(f"\n" + "="*60)
    print(f"🧪 TESTING: {model_name}")
    print(f"   Architecture: {ModelClass.__name__}")
    print(f"   Smoothing: {'ON' if smooth_on else 'OFF'}")
    print("="*60)

    # 1. Load Model
    model = ModelClass(embed_dim=128).to(DEVICE)
    try:
        ckpt = torch.load(model_path, map_location=DEVICE)
        # Handle different saving formats
        if 'model' in ckpt: model.load_state_dict(ckpt['model'])
        else: model.load_state_dict(ckpt)
    except Exception as e:
        print(f"❌ Failed to load {model_name}: {e}")
        continue

    # 2. Build DB (Specific to this model's weights)
    db_tensor, db_meta = build_db_for_model(model, PITCH_DIR, smooth_db=smooth_on)
    if db_tensor is None:
        print("❌ DB Build Failed")
        continue

    # 3. Run Eval (Clean, Soft, Hard, Ultra)
    res, total = run_evaluation(model, db_tensor, db_meta, PITCH_DIR, smooth_query=smooth_on)

    # 4. Print & Store Results
    print(f"\n📊 RESULTS for {model_name} (Total Trials: {total})")
    for mode in ['Clean', 'Soft', 'Hard', 'Ultra']:
        t1 = res[mode]['top1'] / total * 100
        t5 = res[mode]['top5'] / total * 100
        t10 = res[mode]['top10'] / total * 100
        print(f"   {mode:5} | Top-1: {t1:.1f}% | Top-5: {t5:.1f}% | Top-10: {t10:.1f}%")

    final_report[model_name] = res

print("\n✅ GLOBAL EVALUATION COMPLETE.")

🚀 STARTING MULTI-MODEL EVALUATION
📂 Pitch Dir: /content/all_pitch
🎮 Device: cuda

🧪 TESTING: OldCNN_NoSmooth.pth
   Architecture: OldCNN
   Smoothing: OFF


Eval Loop: 100%|██████████| 1616/1616 [02:56<00:00,  9.18it/s]



📊 RESULTS for OldCNN_NoSmooth.pth (Total Trials: 4848)
   Clean | Top-1: 8.6% | Top-5: 25.6% | Top-10: 36.2%
   Soft  | Top-1: 6.7% | Top-5: 20.7% | Top-10: 31.1%
   Hard  | Top-1: 1.9% | Top-5: 5.8% | Top-10: 8.7%
   Ultra | Top-1: 0.6% | Top-5: 1.6% | Top-10: 3.0%

🧪 TESTING: OldCNN_Smooth.pth
   Architecture: OldCNN
   Smoothing: ON


Eval Loop: 100%|██████████| 1616/1616 [02:55<00:00,  9.20it/s]



📊 RESULTS for OldCNN_Smooth.pth (Total Trials: 4848)
   Clean | Top-1: 9.5% | Top-5: 29.1% | Top-10: 40.7%
   Soft  | Top-1: 8.1% | Top-5: 27.3% | Top-10: 39.2%
   Hard  | Top-1: 3.8% | Top-5: 15.4% | Top-10: 24.9%
   Ultra | Top-1: 0.4% | Top-5: 1.7% | Top-10: 2.7%

🧪 TESTING: DeepCNN_NoSmooth.pth
   Architecture: DeepCNN
   Smoothing: OFF


...


📊 RESULTS for DeepCNN_Smooth.pth (Total Trials: 4848)
   Clean | Top-1: 15.3% | Top-5: 33.4% | Top-10: 43.9%
   Soft  | Top-1: 14.0% | Top-5: 32.6% | Top-10: 43.0%
   Hard  | Top-1: 5.3% | Top-5: 19.7% | Top-10: 30.8%
   Ultra | Top-1: 0.9% | Top-5: 2.6% | Top-10: 4.3%

🧪 TESTING: CRNN_NoSmooth.pth
   Architecture: CRNN
   Smoothing: OFF


Eval Loop: 100%|██████████| 1616/1616 [04:02<00:00,  6.65it/s]



📊 RESULTS for CRNN_NoSmooth.pth (Total Trials: 4848)
   Clean | Top-1: 10.9% | Top-5: 29.5% | Top-10: 39.0%
   Soft  | Top-1: 9.9% | Top-5: 28.4% | Top-10: 38.4%
   Hard  | Top-1: 4.2% | Top-5: 17.1% | Top-10: 27.1%
   Ultra | Top-1: 1.1% | Top-5: 4.4% | Top-10: 7.2%

🧪 TESTING: CRNN_Smooth.pth
   Architecture: CRNN
   Smoothing: ON


Eval Loop: 100%|██████████| 1616/1616 [03:55<00:00,  6.85it/s]


📊 RESULTS for CRNN_Smooth.pth (Total Trials: 4848)
   Clean | Top-1: 13.5% | Top-5: 32.9% | Top-10: 42.9%
   Soft  | Top-1: 11.8% | Top-5: 32.0% | Top-10: 42.1%
   Hard  | Top-1: 6.1% | Top-5: 23.1% | Top-10: 34.7%
   Ultra | Top-1: 1.3% | Top-5: 4.6% | Top-10: 7.7%

✅ GLOBAL EVALUATION COMPLETE.


🚀 STARTING MULTI-MODEL EVALUATION
📂 Pitch Dir: /content/all_pitch
🎮 Device: cuda

============================================================
🧪 TESTING: OldCNN_NoSmooth.pth
   Architecture: OldCNN
   Smoothing: OFF
============================================================
Eval Loop: 100%|██████████| 1616/1616 [02:56<00:00,  9.18it/s]

📊 RESULTS for OldCNN_NoSmooth.pth (Total Trials: 4848)
   Clean | Top-1: 8.6% | Top-5: 25.6% | Top-10: 36.2%
   Soft  | Top-1: 6.7% | Top-5: 20.7% | Top-10: 31.1%
   Hard  | Top-1: 1.9% | Top-5: 5.8% | Top-10: 8.7%
   Ultra | Top-1: 0.6% | Top-5: 1.6% | Top-10: 3.0%

============================================================
🧪 TESTING: OldCNN_Smooth.pth
   Architecture: OldCNN
   Smoothing: ON
============================================================
Eval Loop: 100%|██████████| 1616/1616 [02:55<00:00,  9.20it/s]

📊 RESULTS for OldCNN_Smooth.pth (Total Trials: 4848)
   Clean | Top-1: 9.5% | Top-5: 29.1% | Top-10: 40.7%
   Soft  | Top-1: 8.1% | Top-5: 27.3% | Top-10: 39.2%
   Hard  | Top-1: 3.8% | Top-5: 15.4% | Top-10: 24.9%
   Ultra | Top-1: 0.4% | Top-5: 1.7% | Top-10: 2.7%

============================================================
🧪 TESTING: DeepCNN_NoSmooth.pth
   Architecture: DeepCNN
   Smoothing: OFF
============================================================
Eval Loop: 100%|██████████| 1616/1616 [02:59<00:00,  8.99it/s]

📊 RESULTS for DeepCNN_NoSmooth.pth (Total Trials: 4848)
   Clean | Top-1: 13.7% | Top-5: 34.2% | Top-10: 44.9%
   Soft  | Top-1: 13.0% | Top-5: 33.7% | Top-10: 44.8%
   Hard  | Top-1: 4.5% | Top-5: 17.4% | Top-10: 28.0%
   Ultra | Top-1: 0.9% | Top-5: 2.9% | Top-10: 4.4%

============================================================
🧪 TESTING: DeepCNN_Smooth.pth
   Architecture: DeepCNN
   Smoothing: ON
============================================================
Eval Loop: 100%|██████████| 1616/1616 [03:07<00:00,  8.62it/s]

📊 RESULTS for DeepCNN_Smooth.pth (Total Trials: 4848)
   Clean | Top-1: 15.3% | Top-5: 33.4% | Top-10: 43.9%
   Soft  | Top-1: 14.0% | Top-5: 32.6% | Top-10: 43.0%
   Hard  | Top-1: 5.3% | Top-5: 19.7% | Top-10: 30.8%
   Ultra | Top-1: 0.9% | Top-5: 2.6% | Top-10: 4.3%

============================================================
🧪 TESTING: CRNN_NoSmooth.pth
   Architecture: CRNN
   Smoothing: OFF
============================================================
Eval Loop: 100%|██████████| 1616/1616 [04:02<00:00,  6.65it/s]

📊 RESULTS for CRNN_NoSmooth.pth (Total Trials: 4848)
   Clean | Top-1: 10.9% | Top-5: 29.5% | Top-10: 39.0%
   Soft  | Top-1: 9.9% | Top-5: 28.4% | Top-10: 38.4%
   Hard  | Top-1: 4.2% | Top-5: 17.1% | Top-10: 27.1%
   Ultra | Top-1: 1.1% | Top-5: 4.4% | Top-10: 7.2%

============================================================
🧪 TESTING: CRNN_Smooth.pth
   Architecture: CRNN
   Smoothing: ON
============================================================
Eval Loop: 100%|██████████| 1616/1616 [03:55<00:00,  6.85it/s]
📊 RESULTS for CRNN_Smooth.pth (Total Trials: 4848)
   Clean | Top-1: 13.5% | Top-5: 32.9% | Top-10: 42.9%
   Soft  | Top-1: 11.8% | Top-5: 32.0% | Top-10: 42.1%
   Hard  | Top-1: 6.1% | Top-5: 23.1% | Top-10: 34.7%
   Ultra | Top-1: 1.3% | Top-5: 4.6% | Top-10: 7.7%

✅ GLOBAL EVALUATION COMPLETE.



*TRAINING THE TOP 2 MODELS ON A BROADER TRAINING SET*

loading dataset

In [ ]:
# ==============================================================================
# 1. SETUP & DATA MERGING
# ==============================================================================
import os
import subprocess
import sys
import shutil
import glob
from tqdm import tqdm

# Install Deps
print("📦 Installing Dependencies...")
pkgs = ["torchcrepe", "librosa", "torchaudio", "tqdm"]
for p in pkgs:
    subprocess.check_call([sys.executable, "-m", "pip", "install", p])

import torch
import torchcrepe
import torchaudio
import numpy as np
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# --- CONFIG ---
# Previous Data (Originals)
SOURCE_PITCH_ORIGINALS = "/content/all_pitch"

# New Data Sources (Covers & Extras)
ZIP_COVERS = "/content/drive/MyDrive/FINE_TUNE_V3/train_covers_1300.zip"
DIR_SONGS_1 = "/content/drive/MyDrive/FINE_TUNE_songs"
DIR_SONGS_2 = "/content/drive/MyDrive/FINE_TUNE_V3_test/original"

# Output Dirs
AUDIO_DIR_NEW = "/content/expanded_audio_temp" # Temp folder for just the NEW audio
PITCH_DIR_FINAL = "/content/expanded_pitch"    # Final folder for ALL pitch

# --- PREP DIRECTORIES ---
print(f"\n🧹 Setting up directories...")
if os.path.exists(AUDIO_DIR_NEW): shutil.rmtree(AUDIO_DIR_NEW)
if os.path.exists(PITCH_DIR_FINAL): shutil.rmtree(PITCH_DIR_FINAL)
os.makedirs(AUDIO_DIR_NEW, exist_ok=True)
os.makedirs(PITCH_DIR_FINAL, exist_ok=True)

# --- STEP 1: COPY EXISTING ORIGINALS ---
# We verify if the previous step ran. If not, we warn you.
if os.path.exists(SOURCE_PITCH_ORIGINALS) and len(os.listdir(SOURCE_PITCH_ORIGINALS)) > 100:
    print(f"✅ Found {len(os.listdir(SOURCE_PITCH_ORIGINALS))} processed originals.")
    print(f"   Copying to {PITCH_DIR_FINAL}...")
    subprocess.run(f"cp -r '{SOURCE_PITCH_ORIGINALS}/.' '{PITCH_DIR_FINAL}/'", shell=True)
else:
    print("⚠️ WARNING: '/content/all_pitch' is missing or empty!")
    print("   Did you run the previous 'Mega Data Extraction' block?")
    # If you have the zip in Drive, you could add a line here to unzip it.

# --- STEP 2: GATHER NEW AUDIO (Covers) ---
print("\n📂 Gathering NEW Audio (Covers & Extras)...")

# 1. Unzip Covers
if os.path.exists(ZIP_COVERS):
    subprocess.run(f"unzip -q -n '{ZIP_COVERS}' -d '{AUDIO_DIR_NEW}'", shell=True)
    print(f"   Unzipped Covers.")

# 2. Copy Loose Songs
if os.path.exists(DIR_SONGS_1):
    subprocess.run(f"cp -r '{DIR_SONGS_1}/.' '{AUDIO_DIR_NEW}/'", shell=True)
if os.path.exists(DIR_SONGS_2):
    subprocess.run(f"cp -r '{DIR_SONGS_2}/.' '{AUDIO_DIR_NEW}/'", shell=True)

# Flatten
for root, dirs, files in os.walk(AUDIO_DIR_NEW):
    for file in files:
        if file.endswith(".wav") or file.endswith(".mp3"):
            shutil.move(os.path.join(root, file), os.path.join(AUDIO_DIR_NEW, file))

new_files = sorted(glob.glob(os.path.join(AUDIO_DIR_NEW, "*")))
print(f"✅ New Files to Process: {len(new_files)}")

# ==============================================================================
# 2. OPTIMIZED PITCH EXTRACTION (NEW FILES ONLY)
# ==============================================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cpu':
    raise RuntimeError("❌ STOP! Enable GPU.")

print(f"\n🎵 Extracting Pitch for {len(new_files)} NEW files...")

BATCH_SIZE = 2048
for path in tqdm(new_files):
    name = os.path.basename(path)
    if name.endswith(".wav"): name = name.replace(".wav", ".npy")
    elif name.endswith(".mp3"): name = name.replace(".mp3", ".npy")

    save_path = os.path.join(PITCH_DIR_FINAL, name)

    # Skip if we already have this pitch (e.g., from the originals copy)
    if os.path.exists(save_path): continue

    try:
        # Load
        audio, sr = torchaudio.load(path)
        audio = audio.to(device)

        # Resample
        if sr != 16000:
            audio = torchaudio.functional.resample(audio, sr, 16000)
        if audio.shape[0] > 1:
            audio = audio.mean(dim=0, keepdim=True)

        # Predict (Argmax)
        pitch = torchcrepe.predict(
            audio, sample_rate=16000, hop_length=160,
            fmin=50, fmax=2000, model='tiny',
            decoder=torchcrepe.decode.argmax,
            batch_size=BATCH_SIZE, device=device
        )

        # Save
        pitch = pitch.squeeze().cpu().numpy()
        pitch = np.log1p(pitch)
        np.save(save_path, pitch.astype(np.float32))

    except Exception as e:
        print(f"❌ Error on {name}: {e}")

total_count = len(os.listdir(PITCH_DIR_FINAL))
print(f"\n✅ TOTAL EXPANDED DATASET SIZE: {total_count} files.")

# ==============================================================================
# 3. SAVE DATASET TO DRIVE
# ==============================================================================
print(f"\n📦 Saving Expanded Pitch Dataset to Drive...")
ZIP_NAME = "processed_pitch_vectors_EXPANDED"
shutil.make_archive(f"/content/{ZIP_NAME}", 'zip', PITCH_DIR_FINAL)

DEST = f"/content/drive/MyDrive/FIND_TUNE/{ZIP_NAME}.zip"
shutil.copy(f"/content/{ZIP_NAME}.zip", DEST).

print(f"✅ Saved: {DEST}")

📦 Installing Dependencies...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🧹 Setting up directories...
✅ Found 1616 processed originals.
   Copying to /content/expanded_pitch...

📂 Gathering NEW Audio (Covers & Extras)...
   Unzipped Covers.
✅ New Files to Process: 1527

🎵 Extracting Pitch for 1527 NEW files...


100%|██████████| 1527/1527 [24:42<00:00,  1.03it/s]



✅ TOTAL EXPANDED DATASET SIZE: 2944 files.

📦 Saving Expanded Pitch Dataset to Drive...
✅ Saved: /content/drive/MyDrive/FIND_TUNE/processed_pitch_vectors_EXPANDED.zip


training on new dataset (finetuning the top 2 previous model on new data)

In [ ]:
# ==============================================================================
# 4. FINE-TUNE SETUP (FIXED: REMOVED VERBOSE ARG)
# ==============================================================================
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import scipy.signal as sg
import glob
import os
import numpy as np
import torch

# --- CONFIG ---
PITCH_DIR = "/content/expanded_pitch"
MODEL_DIR = "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/testing_models"
SAVE_DIR = "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/finetuned_models"
os.makedirs(SAVE_DIR, exist_ok=True)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- DATASET CLASS ---
class PitchDataset(Dataset):
    def __init__(self, pitch_dir):
        self.files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        # Anchor
        anchor = np.load(self.files[idx])
        # Negative (Random other song)
        neg_idx = random.randint(0, len(self.files)-1)
        while neg_idx == idx: neg_idx = random.randint(0, len(self.files)-1)
        neg = np.load(self.files[neg_idx])

        # Smooth
        anchor = sg.medfilt(anchor, 5).astype(np.float32)
        neg = sg.medfilt(neg, 5).astype(np.float32)

        # Crop 300
        def get_crop(arr):
            if len(arr) < 300: return np.pad(arr, (0, 300-len(arr)), 'constant')
            start = random.randint(0, len(arr)-300)
            return arr[start:start+300]

        a_crop = get_crop(anchor)
        n_crop = get_crop(neg)

        # Augment Anchor -> Positive
        # We make the positive slightly harder here to simulate a cover
        p_crop = a_crop.copy()
        p_crop += np.random.normal(0, 0.06, 300) # Slightly more noise for robustness

        return (
            torch.from_numpy(a_crop).float().unsqueeze(0),
            torch.from_numpy(p_crop).float().unsqueeze(0),
            torch.from_numpy(n_crop).float().unsqueeze(0)
        )

# --- ARCHITECTURES (Must match saved weights) ---
class DeepCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, 5, padding=2), nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, 128))
    def forward_one(self, x): return F.normalize(self.fc(self.cnn(x).squeeze(-1)), p=2, dim=1)

class CRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
        )
        self.lstm = nn.LSTM(256, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, 128))
    def forward_one(self, x):
        x = self.cnn(x).permute(0, 2, 1)
        self.lstm.flatten_parameters()
        out, _ = self.lstm(x)
        return F.normalize(self.fc(torch.mean(out, dim=1)), p=2, dim=1)

# --- TRAIN LOOP ---
def finetune_model(model_name, ModelClass, epochs=60):
    print(f"\n🚀 Fine-Tuning: {model_name}")
    print(f"   Configs: Epochs=60 | Start LR=0.0001 | Patience=4")

    # 1. Load Pre-trained
    model = ModelClass().to(DEVICE)
    path = os.path.join(MODEL_DIR, model_name)
    if not os.path.exists(path):
        print(f"❌ Skipping {model_name} (Not found)")
        return

    ckpt = torch.load(path, map_location=DEVICE)
    if 'model' in ckpt: model.load_state_dict(ckpt['model'])
    else: model.load_state_dict(ckpt)
    print("   ✅ Weights Loaded")

    # 2. Setup
    dataset = PitchDataset(PITCH_DIR)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    # OPTIMIZER STARTING AT 0.0001
    optim = torch.optim.Adam(model.parameters(), lr=0.0001)

    # ADAPTIVE SCHEDULER (FIXED: Removed verbose=True)
    scheduler = ReduceLROnPlateau(optim, mode='min', factor=0.5, patience=4)

    loss_fn = nn.TripletMarginLoss(margin=0.85, p=2)

    # 3. Loop
    model.train()
    best_loss = float('inf')
    save_path = "" # Init variable

    for ep in range(epochs):
        total_loss = 0
        for a, p, n in loader:
            a, p, n = a.to(DEVICE), p.to(DEVICE), n.to(DEVICE)
            optim.zero_grad()
            loss = loss_fn(model.forward_one(a), model.forward_one(p), model.forward_one(n))
            loss.backward()
            optim.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(loader)
        current_lr = optim.param_groups[0]['lr']
        print(f"   Epoch {ep+1}/{epochs} | Loss: {avg_loss:.4f} | LR: {current_lr:.6f}")

        # Step the scheduler based on loss
        scheduler.step(avg_loss)

        # Save Best Checkpoint
        if avg_loss < best_loss:
            best_loss = avg_loss
            save_path = os.path.join(SAVE_DIR, f"FINETUNED_{model_name}")
            torch.save(model.state_dict(), save_path)

    print(f"✅ Finished. Best Loss: {best_loss:.4f}")
    print(f"   Saved to: {save_path}")

# --- EXECUTE ---
finetune_model("DeepCNN_Smooth.pth", DeepCNN)
finetune_model("CRNN_Smooth.pth", CRNN)


🚀 Fine-Tuning: DeepCNN_Smooth.pth
   Configs: Epochs=60 | Start LR=0.0001 | Patience=4
   ✅ Weights Loaded
   Epoch 1/60 | Loss: 0.0133 | LR: 0.000100
   Epoch 2/60 | Loss: 0.0074 | LR: 0.000100
   Epoch 3/60 | Loss: 0.0072 | LR: 0.000100
   Epoch 4/60 | Loss: 0.0059 | LR: 0.000100
   Epoch 5/60 | Loss: 0.0061 | LR: 0.000100
   Epoch 6/60 | Loss: 0.0056 | LR: 0.000100
   Epoch 7/60 | Loss: 0.0056 | LR: 0.000100
   Epoch 8/60 | Loss: 0.0039 | LR: 0.000100
   Epoch 9/60 | Loss: 0.0052 | LR: 0.000100
   Epoch 10/60 | Loss: 0.0054 | LR: 0.000100
   Epoch 11/60 | Loss: 0.0051 | LR: 0.000100
   Epoch 12/60 | Loss: 0.0037 | LR: 0.000100
   Epoch 13/60 | Loss: 0.0047 | LR: 0.000100
   Epoch 14/60 | Loss: 0.0032 | LR: 0.000100
   Epoch 15/60 | Loss: 0.0035 | LR: 0.000100
   Epoch 16/60 | Loss: 0.0038 | LR: 0.000100
   Epoch 17/60 | Loss: 0.0043 | LR: 0.000100
   Epoch 18/60 | Loss: 0.0035 | LR: 0.000100
   Epoch 19/60 | Loss: 0.0039 | LR: 0.000100
   Epoch 20/60 | Loss: 0.0026 | LR: 0.000050
 

EVALUATING THE NOW FINETUNED MODELS

In [ ]:
# ==============================================================================
# 5. EVALUATION OF FINE-TUNED MODELS
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.signal as sg
import os
import glob
import random
from collections import defaultdict
from tqdm import tqdm

# --- CONFIG ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data: The Giant Expanded Dataset
PITCH_DIR = "/content/expanded_pitch"

# Models: The New Fine-Tuned Weights
MODEL_DIR = "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/finetuned_models"

# --- ARCHITECTURES (Must match training exactly) ---
class DeepCNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, 5, padding=2), nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, embed_dim))
    def forward_one(self, x): return F.normalize(self.fc(self.cnn(x).squeeze(-1)), p=2, dim=1)

class CRNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
        )
        self.lstm = nn.LSTM(256, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, embed_dim))
    def forward_one(self, x):
        x = self.cnn(x).permute(0, 2, 1)
        self.lstm.flatten_parameters()
        out, _ = self.lstm(x)
        return F.normalize(self.fc(torch.mean(out, dim=1)), p=2, dim=1)

# --- HELPER FUNCTIONS ---
def smooth_pitch(pitch):
    return sg.medfilt(pitch, kernel_size=5).astype(np.float32)

def augment_clip(arr, mode='clean'):
    arr = arr.copy()
    if mode == 'clean': return arr

    elif mode == 'soft': # Light Noise
        return arr + np.random.normal(0, 0.02, size=len(arr))

    elif mode == 'hard': # Noise + Key Shift + Time Warp
        arr += np.random.normal(0, 0.06, size=len(arr))
        arr[arr > 0] += np.random.uniform(-3, 3) * 0.057
        if random.random() < 0.8: # Warp
            rate = np.random.uniform(0.85, 1.15)
            new_idx = np.linspace(0, len(arr)-1, int(len(arr)*rate))
            arr = np.interp(new_idx, np.arange(len(arr)), arr)
            if len(arr) < 300: arr = np.pad(arr, (0, 300-len(arr)), 'constant')
            else: arr = arr[:300]
        return arr

    elif mode == 'ultra': # The Stress Test
        arr += np.random.normal(0, 0.1, size=len(arr))
        # Cumulative drift (tempo drift)
        rate = np.cumsum(np.linspace(0.8, 1.2, len(arr)))
        rate = rate / rate[-1] * (len(arr)-1)
        arr = np.interp(np.arange(len(arr)), rate, arr)
        return arr
    return arr

def build_db_for_model(model, pitch_dir):
    files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))
    db_embeds = []
    metadata = []

    model.eval()
    with torch.no_grad():
        for f in tqdm(files, desc="Building DB", leave=False):
            arr = np.load(f)
            arr = smooth_pitch(arr) # Always smooth DB tracks

            # Windowing (3s windows, 1.5s hop)
            windows = []
            offsets = []
            i = 0
            while i + 300 <= len(arr):
                crop = arr[i:i+300]
                if np.mean(crop > 0) > 0.1: # Skip silence
                    windows.append(crop)
                    offsets.append(i/100.0)
                i += 150

            if not windows: continue

            # Batch Inference
            w_tensor = torch.from_numpy(np.stack(windows)).float().unsqueeze(1).to(DEVICE)
            # Batch processing to avoid OOM on huge files
            batch_size = 512
            for k in range(0, len(w_tensor), batch_size):
                batch = w_tensor[k:k+batch_size]
                embeddings = model.forward_one(batch)
                db_embeds.append(embeddings)

            song_name = os.path.basename(f).replace(".npy","")
            for t in offsets:
                metadata.append((song_name, t))

    if not db_embeds: return None, None
    return torch.cat(db_embeds), metadata

def run_evaluation(model, db_tensor, db_meta, pitch_dir):
    files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))
    # Pick random subset of 300 songs for evaluation speed
    # (Checking all 1600+ takes too long, 300 is statistically significant)
    if len(files) > 300:
        random.shuffle(files)
        files = files[:300]

    modes = ['Clean', 'Soft', 'Hard', 'Ultra']
    results = {m: {'top1':0, 'top5':0, 'top10':0} for m in modes}
    total_clips = 0

    model.eval()

    for f in tqdm(files, desc="Eval Loop"):
        full_arr = np.load(f)
        if len(full_arr) < 500: continue
        target_name = os.path.basename(f).replace(".npy","")

        # Take 3 random starting points per song
        starts = np.random.randint(0, len(full_arr)-300, 3)

        for s in starts:
            base_clip = full_arr[s:s+300]
            base_clip = smooth_pitch(base_clip) # Always smooth query

            total_clips += 1

            for mode in modes:
                aug_clip = augment_clip(base_clip, mode=mode.lower())
                q_tensor = torch.from_numpy(aug_clip).float().unsqueeze(0).unsqueeze(0).to(DEVICE)

                with torch.no_grad():
                    q_emb = model.forward_one(q_tensor)

                # Geometric Scoring
                dists = torch.cdist(q_emb, db_tensor)
                vals, inds = torch.topk(dists, k=50, largest=False)

                votes = defaultdict(float)
                for k in range(50):
                    idx = inds[0, k].item()
                    dist = vals[0, k].item()
                    m_name, m_time = db_meta[idx]
                    score = 1.0 / (dist + 1e-4)
                    votes[m_name] += score

                ranked = sorted(votes.items(), key=lambda x: x[1], reverse=True)
                top_names = [x[0] for x in ranked]

                if not top_names: continue

                if top_names[0] == target_name: results[mode]['top1'] += 1
                if target_name in top_names[:5]: results[mode]['top5'] += 1
                if target_name in top_names[:10]: results[mode]['top10'] += 1

    return results, total_clips

# ==============================================================================
# MASTER LOOP (FINE-TUNED MODELS)
# ==============================================================================
experiments = [
    ("FINETUNED_DeepCNN_Smooth.pth", DeepCNN),
    ("FINETUNED_CRNN_Smooth.pth",    CRNN),
]

print(f"🚀 STARTING FINE-TUNED MODEL EVALUATION")
print(f"📂 Pitch Dir: {PITCH_DIR}")
print(f"📂 Model Dir: {MODEL_DIR}")

for model_name, ModelClass in experiments:
    model_path = os.path.join(MODEL_DIR, model_name)

    if not os.path.exists(model_path):
        print(f"\n⚠️ SKIPPING {model_name} (File not found)")
        continue

    print(f"\n" + "="*60)
    print(f"🧪 TESTING: {model_name}")
    print("="*60)

    # 1. Load
    model = ModelClass(embed_dim=128).to(DEVICE)
    try:
        ckpt = torch.load(model_path, map_location=DEVICE)
        if 'model' in ckpt: model.load_state_dict(ckpt['model'])
        else: model.load_state_dict(ckpt)
    except Exception as e:
        print(f"❌ Failed to load: {e}")
        continue

    # 2. Build DB
    db_tensor, db_meta = build_db_for_model(model, PITCH_DIR)
    if db_tensor is None: continue

    # 3. Eval
    res, total = run_evaluation(model, db_tensor, db_meta, PITCH_DIR)

    # 4. Report
    print(f"\n📊 RESULTS for {model_name} (Total Trials: {total})")
    for mode in ['Clean', 'Soft', 'Hard', 'Ultra']:
        t1 = res[mode]['top1'] / total * 100
        t5 = res[mode]['top5'] / total * 100
        t10 = res[mode]['top10'] / total * 100
        print(f"   {mode:5} | Top-1: {t1:.1f}% | Top-5: {t5:.1f}% | Top-10: {t10:.1f}%")

print("\n✅ DONE.")

🚀 STARTING FINE-TUNED MODEL EVALUATION
📂 Pitch Dir: /content/expanded_pitch
📂 Model Dir: /content/drive/MyDrive/FIND_TUNE/pitch_based_model/finetuned_models

🧪 TESTING: FINETUNED_DeepCNN_Smooth.pth


Eval Loop: 100%|██████████| 300/300 [00:47<00:00,  6.29it/s]



📊 RESULTS for FINETUNED_DeepCNN_Smooth.pth (Total Trials: 900)
   Clean | Top-1: 23.8% | Top-5: 49.4% | Top-10: 62.9%
   Soft  | Top-1: 23.1% | Top-5: 49.7% | Top-10: 63.2%
   Hard  | Top-1: 5.0% | Top-5: 13.0% | Top-10: 19.1%
   Ultra | Top-1: 10.3% | Top-5: 35.0% | Top-10: 50.2%

🧪 TESTING: FINETUNED_CRNN_Smooth.pth


Eval Loop: 100%|██████████| 300/300 [00:57<00:00,  5.21it/s]


📊 RESULTS for FINETUNED_CRNN_Smooth.pth (Total Trials: 900)
   Clean | Top-1: 21.7% | Top-5: 48.3% | Top-10: 62.7%
   Soft  | Top-1: 21.6% | Top-5: 49.0% | Top-10: 61.8%
   Hard  | Top-1: 6.2% | Top-5: 19.7% | Top-10: 27.4%
   Ultra | Top-1: 12.2% | Top-5: 39.7% | Top-10: 54.7%

✅ DONE.


============================================================
🧪 TESTING: FINETUNED_DeepCNN_Smooth.pth
============================================================
Eval Loop: 100%|██████████| 300/300 [00:48<00:00,  6.20it/s]

📊 RESULTS for FINETUNED_DeepCNN_Smooth.pth (Total Trials: 900)
   Clean | Top-1: 22.8% | Top-5: 52.3% | Top-10: 62.9%
   Soft  | Top-1: 23.2% | Top-5: 52.4% | Top-10: 63.0%
   Hard  | Top-1: 4.9% | Top-5: 14.6% | Top-10: 21.1%
   Ultra | Top-1: 12.1% | Top-5: 36.4% | Top-10: 52.0%

============================================================
🧪 TESTING: FINETUNED_CRNN_Smooth.pth
============================================================
Eval Loop: 100%|██████████| 300/300 [00:58<00:00,  5.16it/s]
📊 RESULTS for FINETUNED_CRNN_Smooth.pth (Total Trials: 900)
   Clean | Top-1: 20.0% | Top-5: 49.1% | Top-10: 62.8%
   Soft  | Top-1: 18.6% | Top-5: 48.4% | Top-10: 61.8%
   Hard  | Top-1: 5.0% | Top-5: 17.6% | Top-10: 26.0%
   Ultra | Top-1: 9.6% | Top-5: 37.3% | Top-10: 55.1%


A MORE DYNAMIC FINETUNING HERE WE MAKE POSITIVE PAIR OF THE ANCHOR CLIP ON DIFF AUGMENTATION

In [ ]:
# ==============================================================================
# MULTI-TASK FINE-TUNING (TEMPORAL + SOFT + HARD + ULTRA)
# ==============================================================================
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import scipy.signal as sg
import glob
import os
import numpy as np
import torch

# --- CONFIG ---
PITCH_DIR = "/content/expanded_pitch"
MODEL_DIR = "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/testing_models"
SAVE_DIR = "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/finetuned_models"
os.makedirs(SAVE_DIR, exist_ok=True)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TARGET_LEN = 300

# --- AUGMENTATION LOGIC ---
def get_crop_and_pad(arr, target_len=300):
    if len(arr) < target_len:
        return np.pad(arr, (0, target_len - len(arr)), 'constant')
    start = random.randint(0, len(arr) - target_len)
    return arr[start:start + target_len]

def augment_pitch(arr, mode):
    arr = arr.copy()

    if mode == 'soft': # Light Noise
        arr += np.random.normal(0, 0.02, size=len(arr))

    elif mode == 'hard': # Jitter + Key Shift
        arr += np.random.normal(0, 0.06, size=len(arr))
        shift = np.random.uniform(-3, 3) * 0.057
        arr[arr > 0] += shift

    elif mode == 'ultra': # Time Warp + Dropout + Heavy Noise
        # 1. Heavy Jitter
        arr += np.random.normal(0, 0.1, size=len(arr))

        # 2. Time Warp (Tempo Drift)
        # We stretch/squash and then re-sample back to target length
        rate = np.random.uniform(0.8, 1.2)
        old_indices = np.arange(len(arr))
        new_length = int(len(arr) * rate)
        new_indices = np.linspace(0, len(arr)-1, new_length)
        arr = np.interp(new_indices, old_indices, arr)

        # 3. Re-Crop/Pad to 300 after warping
        if len(arr) != TARGET_LEN:
            if len(arr) < TARGET_LEN:
                arr = np.pad(arr, (0, TARGET_LEN - len(arr)), 'constant')
            else:
                start = random.randint(0, len(arr) - TARGET_LEN)
                arr = arr[start:start+TARGET_LEN]

        # 4. Dropout (Simulate bad mic connection)
        if random.random() < 0.5:
            drop_len = random.randint(10, 50)
            drop_start = random.randint(0, len(arr) - drop_len)
            arr[drop_start:drop_start+drop_len] = 0

    return arr.astype(np.float32)

# --- MULTI-TASK DATASET ---
class MultiTaskDataset(Dataset):
    def __init__(self, pitch_dir):
        self.files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        # 1. Load Anchor File
        anchor_full = np.load(self.files[idx])

        # 2. Load Negative File
        neg_idx = random.randint(0, len(self.files)-1)
        while neg_idx == idx:
            neg_idx = random.randint(0, len(self.files)-1)
        neg_full = np.load(self.files[neg_idx])

        # 3. Base Crops (Smoothed)
        # Crop A1: The Anchor
        a_raw = get_crop_and_pad(anchor_full)
        # Crop A2: Different part of same song (For Temporal Task)
        p_temporal_raw = get_crop_and_pad(anchor_full)
        # Crop N: Negative
        n_raw = get_crop_and_pad(neg_full)

        # Apply Smoothing
        anchor = sg.medfilt(a_raw, 5).astype(np.float32)
        pos_temporal = sg.medfilt(p_temporal_raw, 5).astype(np.float32)
        neg = sg.medfilt(n_raw, 5).astype(np.float32)

        # 4. Generate Augmented Positives (From the Anchor crop)
        pos_soft  = augment_pitch(anchor, 'soft')
        pos_hard  = augment_pitch(anchor, 'hard')
        pos_ultra = augment_pitch(anchor, 'ultra')

        return (
            torch.from_numpy(anchor).float().unsqueeze(0),       # Anchor
            torch.from_numpy(pos_temporal).float().unsqueeze(0), # Task 1
            torch.from_numpy(pos_soft).float().unsqueeze(0),     # Task 2
            torch.from_numpy(pos_hard).float().unsqueeze(0),     # Task 3
            torch.from_numpy(pos_ultra).float().unsqueeze(0),    # Task 4
            torch.from_numpy(neg).float().unsqueeze(0)           # Negative
        )

# --- ARCHITECTURES ---
class DeepCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, 5, padding=2), nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, 128))
    def forward_one(self, x): return F.normalize(self.fc(self.cnn(x).squeeze(-1)), p=2, dim=1)

class CRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
        )
        self.lstm = nn.LSTM(256, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, 128))
    def forward_one(self, x):
        x = self.cnn(x).permute(0, 2, 1)
        self.lstm.flatten_parameters()
        out, _ = self.lstm(x)
        return F.normalize(self.fc(torch.mean(out, dim=1)), p=2, dim=1)

# --- MULTI-TASK TRAINING LOOP ---
def train_multitask(model_name, ModelClass, epochs=60):
    print(f"\n🚀 Multi-Task Training: {model_name}")
    print(f"   Tasks: Temporal + Soft + Hard + Ultra")

    # 1. Load Base Model
    model = ModelClass().to(DEVICE)
    path = os.path.join(MODEL_DIR, model_name)
    if not os.path.exists(path):
        print(f"❌ Skipping {model_name} (Not found)")
        return

    ckpt = torch.load(path, map_location=DEVICE)
    if 'model' in ckpt: model.load_state_dict(ckpt['model'])
    else: model.load_state_dict(ckpt)
    print("   ✅ Base Weights Loaded")

    # 2. Setup
    dataset = MultiTaskDataset(PITCH_DIR)
    loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

    optim = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = ReduceLROnPlateau(optim, mode='min', factor=0.5, patience=4)

    # Using slightly stricter margin for Hard/Ultra to force separation
    loss_fn = nn.TripletMarginLoss(margin=0.85, p=2)

    # 3. Loop
    model.train()
    best_loss = float('inf')
    save_path = ""

    for ep in range(epochs):
        total_loss = 0
        for batch in loader:
            # Unpack all 6 tensors
            # anchor, p_temp, p_soft, p_hard, p_ultra, neg
            anch, p1, p2, p3, p4, neg = [t.to(DEVICE) for t in batch]

            optim.zero_grad()

            # Forward Pass (Shared Anchor, Shared Negative)
            a_emb = model.forward_one(anch)
            n_emb = model.forward_one(neg)

            # 4x Positive Forward Passes
            p1_emb = model.forward_one(p1) # Temporal
            p2_emb = model.forward_one(p2) # Soft
            p3_emb = model.forward_one(p3) # Hard
            p4_emb = model.forward_one(p4) # Ultra

            # Calculate 4 Losses
            loss1 = loss_fn(a_emb, p1_emb, n_emb)
            loss2 = loss_fn(a_emb, p2_emb, n_emb)
            loss3 = loss_fn(a_emb, p3_emb, n_emb)
            loss4 = loss_fn(a_emb, p4_emb, n_emb)

            # Weighted Sum (You can tune weights if needed, equal is usually fine)
            combined_loss = loss1 + loss2 + loss3 + loss4

            combined_loss.backward()
            optim.step()

            # We track the average individual loss, so divide by 4
            total_loss += (combined_loss.item() / 4.0)

        avg_loss = total_loss / len(loader)
        current_lr = optim.param_groups[0]['lr']
        print(f"   Epoch {ep+1}/{epochs} | Avg Loss: {avg_loss:.4f} | LR: {current_lr:.6f}")

        scheduler.step(avg_loss)

        if avg_loss < best_loss:
            best_loss = avg_loss
            save_path = os.path.join(SAVE_DIR, f"MULTITASK_{model_name}")
            torch.save(model.state_dict(), save_path)

    print(f"✅ Finished. Best Loss: {best_loss:.4f}")
    print(f"   Saved to: {save_path}")

# --- EXECUTE ---
train_multitask("DeepCNN_Smooth.pth", DeepCNN)
train_multitask("CRNN_Smooth.pth", CRNN)


🚀 Multi-Task Training: DeepCNN_Smooth.pth
   Tasks: Temporal + Soft + Hard + Ultra
   ✅ Base Weights Loaded
   Epoch 1/60 | Avg Loss: 0.2117 | LR: 0.000100
   Epoch 2/60 | Avg Loss: 0.1916 | LR: 0.000100
   Epoch 3/60 | Avg Loss: 0.1851 | LR: 0.000100
   Epoch 4/60 | Avg Loss: 0.1843 | LR: 0.000100
   Epoch 5/60 | Avg Loss: 0.1828 | LR: 0.000100
   Epoch 6/60 | Avg Loss: 0.1838 | LR: 0.000100
   Epoch 7/60 | Avg Loss: 0.1839 | LR: 0.000100
   Epoch 8/60 | Avg Loss: 0.1814 | LR: 0.000100
   Epoch 9/60 | Avg Loss: 0.1804 | LR: 0.000100
   Epoch 10/60 | Avg Loss: 0.1782 | LR: 0.000100
   Epoch 11/60 | Avg Loss: 0.1782 | LR: 0.000100
   Epoch 12/60 | Avg Loss: 0.1821 | LR: 0.000100
   Epoch 13/60 | Avg Loss: 0.1822 | LR: 0.000100
   Epoch 14/60 | Avg Loss: 0.1805 | LR: 0.000100
   Epoch 15/60 | Avg Loss: 0.1804 | LR: 0.000100
   Epoch 16/60 | Avg Loss: 0.1736 | LR: 0.000050
   Epoch 17/60 | Avg Loss: 0.1758 | LR: 0.000050
   Epoch 18/60 | Avg Loss: 0.1825 | LR: 0.000050
   Epoch 19/60 | A

EVAL

In [ ]:
# ==============================================================================
# 6. EVALUATION OF MULTI-TASK MODELS
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.signal as sg
import os
import glob
import random
from collections import defaultdict
from tqdm import tqdm

# --- CONFIG ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data: The Giant Expanded Dataset
PITCH_DIR = "/content/expanded_pitch"

# Models: Point to where you saved the MULTITASK models
MODEL_DIR = "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/finetuned_models"

# --- ARCHITECTURES (Must match training exactly) ---
class DeepCNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, 5, padding=2), nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, embed_dim))
    def forward_one(self, x): return F.normalize(self.fc(self.cnn(x).squeeze(-1)), p=2, dim=1)

class CRNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
        )
        self.lstm = nn.LSTM(256, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, embed_dim))
    def forward_one(self, x):
        x = self.cnn(x).permute(0, 2, 1)
        self.lstm.flatten_parameters()
        out, _ = self.lstm(x)
        return F.normalize(self.fc(torch.mean(out, dim=1)), p=2, dim=1)

# --- HELPER FUNCTIONS ---
def smooth_pitch(pitch):
    return sg.medfilt(pitch, kernel_size=5).astype(np.float32)

def augment_clip(arr, mode='clean'):
    arr = arr.copy()
    if mode == 'clean': return arr

    elif mode == 'soft': # Light Noise
        return arr + np.random.normal(0, 0.02, size=len(arr))

    elif mode == 'hard': # Noise + Key Shift + Time Warp
        arr += np.random.normal(0, 0.06, size=len(arr))
        arr[arr > 0] += np.random.uniform(-3, 3) * 0.057
        if random.random() < 0.8: # Warp
            rate = np.random.uniform(0.85, 1.15)
            new_idx = np.linspace(0, len(arr)-1, int(len(arr)*rate))
            arr = np.interp(new_idx, np.arange(len(arr)), arr)
            if len(arr) < 300: arr = np.pad(arr, (0, 300-len(arr)), 'constant')
            else: arr = arr[:300]
        return arr

    elif mode == 'ultra': # The Stress Test
        arr += np.random.normal(0, 0.1, size=len(arr))
        # Cumulative drift (tempo drift)
        rate = np.cumsum(np.linspace(0.8, 1.2, len(arr)))
        rate = rate / rate[-1] * (len(arr)-1)
        arr = np.interp(np.arange(len(arr)), rate, arr)
        return arr
    return arr

def build_db_for_model(model, pitch_dir):
    files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))
    db_embeds = []
    metadata = []

    model.eval()
    with torch.no_grad():
        for f in tqdm(files, desc="Building DB", leave=False):
            arr = np.load(f)
            arr = smooth_pitch(arr) # Always smooth DB tracks

            # Windowing (3s windows, 1.5s hop)
            windows = []
            offsets = []
            i = 0
            while i + 300 <= len(arr):
                crop = arr[i:i+300]
                if np.mean(crop > 0) > 0.1: # Skip silence
                    windows.append(crop)
                    offsets.append(i/100.0)
                i += 150

            if not windows: continue

            # Batch Inference
            w_tensor = torch.from_numpy(np.stack(windows)).float().unsqueeze(1).to(DEVICE)
            # Batch processing to avoid OOM on huge files
            batch_size = 512
            for k in range(0, len(w_tensor), batch_size):
                batch = w_tensor[k:k+batch_size]
                embeddings = model.forward_one(batch)
                db_embeds.append(embeddings)

            song_name = os.path.basename(f).replace(".npy","")
            for t in offsets:
                metadata.append((song_name, t))

    if not db_embeds: return None, None
    return torch.cat(db_embeds), metadata

def run_evaluation(model, db_tensor, db_meta, pitch_dir):
    files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))
    # Pick random subset of 300 songs for evaluation speed
    # (Checking all 1600+ takes too long, 300 is statistically significant)
    if len(files) > 300:
        random.shuffle(files)
        files = files[:300]

    modes = ['Clean', 'Soft', 'Hard', 'Ultra']
    results = {m: {'top1':0, 'top5':0, 'top10':0} for m in modes}
    total_clips = 0

    model.eval()

    for f in tqdm(files, desc="Eval Loop"):
        full_arr = np.load(f)
        if len(full_arr) < 500: continue
        target_name = os.path.basename(f).replace(".npy","")

        # Take 3 random starting points per song
        starts = np.random.randint(0, len(full_arr)-300, 3)

        for s in starts:
            base_clip = full_arr[s:s+300]
            base_clip = smooth_pitch(base_clip) # Always smooth query

            total_clips += 1

            for mode in modes:
                aug_clip = augment_clip(base_clip, mode=mode.lower())
                q_tensor = torch.from_numpy(aug_clip).float().unsqueeze(0).unsqueeze(0).to(DEVICE)

                with torch.no_grad():
                    q_emb = model.forward_one(q_tensor)

                # Geometric Scoring
                dists = torch.cdist(q_emb, db_tensor)
                vals, inds = torch.topk(dists, k=50, largest=False)

                votes = defaultdict(float)
                for k in range(50):
                    idx = inds[0, k].item()
                    dist = vals[0, k].item()
                    m_name, m_time = db_meta[idx]
                    score = 1.0 / (dist + 1e-4)
                    votes[m_name] += score

                ranked = sorted(votes.items(), key=lambda x: x[1], reverse=True)
                top_names = [x[0] for x in ranked]

                if not top_names: continue

                if top_names[0] == target_name: results[mode]['top1'] += 1
                if target_name in top_names[:5]: results[mode]['top5'] += 1
                if target_name in top_names[:10]: results[mode]['top10'] += 1

    return results, total_clips

# ==============================================================================
# MASTER LOOP (MULTI-TASK MODELS)
# ==============================================================================
# Note: I added the '_v2' suffix based on our previous step.
# If you didn't use the suffix, just remove it from the string here.
experiments = [
    ("MULTITASK_DeepCNN_Smooth.pth", DeepCNN),
    ("MULTITASK_CRNN_Smooth .pth",    CRNN),
]

print(f"🚀 STARTING MULTI-TASK MODEL EVALUATION")
print(f"📂 Pitch Dir: {PITCH_DIR}")
print(f"📂 Model Dir: {MODEL_DIR}")

for model_name, ModelClass in experiments:
    model_path = os.path.join(MODEL_DIR, model_name)

    if not os.path.exists(model_path):
        print(f"\n⚠️ SKIPPING {model_name} (File not found)")
        continue

    print(f"\n" + "="*60)
    print(f"🧪 TESTING: {model_name}")
    print("="*60)

    # 1. Load
    model = ModelClass(embed_dim=128).to(DEVICE)
    try:
        ckpt = torch.load(model_path, map_location=DEVICE)
        if 'model' in ckpt: model.load_state_dict(ckpt['model'])
        else: model.load_state_dict(ckpt)
    except Exception as e:
        print(f"❌ Failed to load: {e}")
        continue

    # 2. Build DB
    db_tensor, db_meta = build_db_for_model(model, PITCH_DIR)
    if db_tensor is None: continue

    # 3. Eval
    res, total = run_evaluation(model, db_tensor, db_meta, PITCH_DIR)

    # 4. Report
    print(f"\n📊 RESULTS for {model_name} (Total Trials: {total})")
    for mode in ['Clean', 'Soft', 'Hard', 'Ultra']:
        t1 = res[mode]['top1'] / total * 100
        t5 = res[mode]['top5'] / total * 100
        t10 = res[mode]['top10'] / total * 100
        print(f"   {mode:5} | Top-1: {t1:.1f}% | Top-5: {t5:.1f}% | Top-10: {t10:.1f}%")

print("\n✅ DONE.")

🚀 STARTING MULTI-TASK MODEL EVALUATION
📂 Pitch Dir: /content/expanded_pitch
📂 Model Dir: /content/drive/MyDrive/FIND_TUNE/pitch_based_model/finetuned_models

🧪 TESTING: MULTITASK_DeepCNN_Smooth.pth


Eval Loop: 100%|██████████| 300/300 [00:47<00:00,  6.27it/s]



📊 RESULTS for MULTITASK_DeepCNN_Smooth.pth (Total Trials: 900)
   Clean | Top-1: 19.7% | Top-5: 42.3% | Top-10: 52.7%
   Soft  | Top-1: 19.7% | Top-5: 43.0% | Top-10: 53.1%
   Hard  | Top-1: 4.4% | Top-5: 13.9% | Top-10: 21.9%
   Ultra | Top-1: 5.2% | Top-5: 17.1% | Top-10: 27.4%

🧪 TESTING: MULTITASK_CRNN_Smooth.pth


Eval Loop: 100%|██████████| 300/300 [00:57<00:00,  5.20it/s]


📊 RESULTS for MULTITASK_CRNN_Smooth.pth (Total Trials: 900)
   Clean | Top-1: 17.2% | Top-5: 38.9% | Top-10: 49.7%
   Soft  | Top-1: 16.2% | Top-5: 39.0% | Top-10: 50.1%
   Hard  | Top-1: 4.9% | Top-5: 14.4% | Top-10: 23.6%
   Ultra | Top-1: 5.0% | Top-5: 16.9% | Top-10: 25.7%

✅ DONE.


============================================================
🧪 TESTING: MULTITASK_DeepCNN_Smooth.pth
============================================================
Eval Loop: 100%|██████████| 300/300 [00:47<00:00,  6.27it/s]

📊 RESULTS for MULTITASK_DeepCNN_Smooth.pth (Total Trials: 900)
   Clean | Top-1: 19.7% | Top-5: 42.3% | Top-10: 52.7%
   Soft  | Top-1: 19.7% | Top-5: 43.0% | Top-10: 53.1%
   Hard  | Top-1: 4.4% | Top-5: 13.9% | Top-10: 21.9%
   Ultra | Top-1: 5.2% | Top-5: 17.1% | Top-10: 27.4%

============================================================
🧪 TESTING: MULTITASK_CRNN_Smooth.pth
============================================================
Eval Loop: 100%|██████████| 300/300 [00:57<00:00,  5.20it/s]
📊 RESULTS for MULTITASK_CRNN_Smooth.pth (Total Trials: 900)
   Clean | Top-1: 17.2% | Top-5: 38.9% | Top-10: 49.7%
   Soft  | Top-1: 16.2% | Top-5: 39.0% | Top-10: 50.1%
   Hard  | Top-1: 4.9% | Top-5: 14.4% | Top-10: 23.6%
   Ultra | Top-1: 5.0% | Top-5: 16.9% | Top-10: 25.7%


BUILDING A MODEL FROM BASE UP

In [ ]:
# ==============================================================================
# FULL RETRAIN (DUAL ITERATION + ADAPTIVE LR SCHEDULER)
# ==============================================================================

def train_from_scratch(model_name, ModelClass):
    print(f"\n🚀 Training From Scratch (Dual Iter): {model_name}")
    # Higher starting LR (0.001) is essential for random weights
    print(f"   Configs: Epochs={EPOCHS} | Start LR=0.001 | Patience=5")

    # 1. Initialize Fresh Model
    model = ModelClass(embed_dim=128).to(DEVICE)

    # 2. Setup
    dataset = PitchDataset(PITCH_DIR)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

    # Optimizer (Per-parameter adaptive learning)
    optim = torch.optim.Adam(model.parameters(), lr=0.001)

    # Scheduler (Global adaptation based on loss plateau)
    # factor=0.5 means it cuts LR in half when it gets stuck
    # patience=5 is your requested tolerance
    scheduler = ReduceLROnPlateau(optim, mode='min', factor=0.5, patience=5)

    loss_fn = nn.TripletMarginLoss(margin=0.85, p=2)

    # 3. Loop
    model.train()
    best_loss = float('inf')
    save_path = os.path.join(SAVE_DIR, f"RETRAINED_{model_name}.pth")

    for ep in range(EPOCHS):
        total_loss = 0
        for batch in loader:
            a, p_hard, p_soft, n = [t.to(DEVICE) for t in batch]

            optim.zero_grad()

            # --- FORWARD PASS (Batch Stacking for 30% Speedup) ---
            combined_input = torch.cat([a, p_hard, p_soft, n], dim=0)
            combined_emb = model.forward_one(combined_input)

            bs = a.size(0)
            a_emb = combined_emb[0:bs]
            ph_emb = combined_emb[bs:2*bs]
            ps_emb = combined_emb[2*bs:3*bs]
            n_emb = combined_emb[3*bs:]

            # --- DUAL LOSS CALCULATION ---
            # 1. Temporal/Hard Loss (Melody Structure)
            loss_structure = loss_fn(a_emb, ph_emb, n_emb)
            # 2. Stability/Soft Loss (Signal Robustness)
            loss_stability = loss_fn(a_emb, ps_emb, n_emb)

            total_batch_loss = loss_structure + loss_stability
            total_batch_loss.backward()

            # Gradient Clipping (Safety for scratch training)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optim.step()
            total_loss += (total_batch_loss.item() / 2.0)

        avg_loss = total_loss / len(loader)

        # --- ADAPTIVE LR STEP ---
        # The scheduler checks if avg_loss has improved
        scheduler.step(avg_loss)

        # Get the current LR to print (from any param group)
        current_lr = optim.param_groups[0]['lr']

        print(f"   Epoch {ep+1}/{EPOCHS} | Avg Loss: {avg_loss:.4f} | LR: {current_lr:.6f}")

        # Save Best Checkpoint
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), save_path)
            # print(f"   ⭐ New Best Loss! Model Saved.")

    print(f"\n✅ Finished. Best Loss: {best_loss:.4f}")
    print(f"   Final Model: {save_path}")

# --- EXECUTE ---
train_from_scratch("DeepCNN_Full_Dual", DeepCNN)
train_from_scratch("CRNN_Full_Dual", CRNN)


🚀 Training From Scratch (Dual Iter): DeepCNN_Full_Dual
   Configs: Epochs=100 | Start LR=0.001 | Patience=5
   Epoch 1/100 | Avg Loss: 0.3486 | LR: 0.001000
   Epoch 2/100 | Avg Loss: 0.3317 | LR: 0.001000
   Epoch 3/100 | Avg Loss: 0.3304 | LR: 0.001000
   Epoch 4/100 | Avg Loss: 0.3258 | LR: 0.001000
   Epoch 5/100 | Avg Loss: 0.3252 | LR: 0.001000
   Epoch 6/100 | Avg Loss: 0.3144 | LR: 0.001000
   Epoch 7/100 | Avg Loss: 0.3163 | LR: 0.001000
   Epoch 8/100 | Avg Loss: 0.3086 | LR: 0.001000
   Epoch 9/100 | Avg Loss: 0.3103 | LR: 0.001000
   Epoch 10/100 | Avg Loss: 0.3116 | LR: 0.001000
   Epoch 11/100 | Avg Loss: 0.3075 | LR: 0.001000
   Epoch 12/100 | Avg Loss: 0.3044 | LR: 0.001000
   Epoch 13/100 | Avg Loss: 0.2965 | LR: 0.001000
   Epoch 14/100 | Avg Loss: 0.2948 | LR: 0.001000
   Epoch 15/100 | Avg Loss: 0.3042 | LR: 0.001000
   Epoch 16/100 | Avg Loss: 0.2985 | LR: 0.001000
   Epoch 17/100 | Avg Loss: 0.2940 | LR: 0.001000
   Epoch 18/100 | Avg Loss: 0.2968 | LR: 0.001000


In [ ]:
# ==============================================================================
# 6. EVALUATION OF RETRAINED MODELS
# ==============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.signal as sg
import os
import glob
import random
from collections import defaultdict
from tqdm import tqdm

# --- CONFIG ---
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data: The Giant Expanded Dataset (3000+ Songs)
PITCH_DIR = "/content/expanded_pitch"

# Models: Point to your new retrained models
MODEL_DIR = "/content/drive/MyDrive/FIND_TUNE/pitch_based_model/retrained_models"

# --- ARCHITECTURES (Must match training exactly) ---
class DeepCNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 32, 5, padding=2), nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, embed_dim))
    def forward_one(self, x): return F.normalize(self.fc(self.cnn(x).squeeze(-1)), p=2, dim=1)

class CRNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 64, 5, padding=2), nn.BatchNorm1d(64), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(64, 128, 3, padding=1), nn.BatchNorm1d(128), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(128, 256, 3, padding=1), nn.BatchNorm1d(256), nn.ReLU(),
        )
        self.lstm = nn.LSTM(256, 128, num_layers=2, batch_first=True, bidirectional=True)
        self.fc = nn.Sequential(nn.Linear(256, 256), nn.ReLU(), nn.Linear(256, embed_dim))
    def forward_one(self, x):
        x = self.cnn(x).permute(0, 2, 1)
        self.lstm.flatten_parameters()
        out, _ = self.lstm(x)
        return F.normalize(self.fc(torch.mean(out, dim=1)), p=2, dim=1)

# --- HELPER FUNCTIONS ---
def smooth_pitch(pitch):
    return sg.medfilt(pitch, kernel_size=5).astype(np.float32)

def augment_clip(arr, mode='clean'):
    arr = arr.copy()
    if mode == 'clean': return arr

    elif mode == 'soft': # Light Noise
        return arr + np.random.normal(0, 0.02, size=len(arr))

    elif mode == 'hard': # Noise + Key Shift + Time Warp (Standard)
        arr += np.random.normal(0, 0.06, size=len(arr))
        arr[arr > 0] += np.random.uniform(-3, 3) * 0.057
        if random.random() < 0.8: # Warp
            rate = np.random.uniform(0.85, 1.15)
            new_idx = np.linspace(0, len(arr)-1, int(len(arr)*rate))
            arr = np.interp(new_idx, np.arange(len(arr)), arr)
            if len(arr) < 300: arr = np.pad(arr, (0, 300-len(arr)), 'constant')
            else: arr = arr[:300]
        return arr

    elif mode == 'ultra': # The Stress Test (Drift + Dropout)
        arr += np.random.normal(0, 0.1, size=len(arr))
        # Cumulative drift (tempo drift)
        rate = np.cumsum(np.linspace(0.8, 1.2, len(arr)))
        rate = rate / rate[-1] * (len(arr)-1)
        arr = np.interp(np.arange(len(arr)), rate, arr)
        return arr
    return arr

def build_db_for_model(model, pitch_dir):
    files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))
    db_embeds = []
    metadata = []

    model.eval()
    print(f"   Building Database from {len(files)} files...")
    with torch.no_grad():
        for f in tqdm(files, desc="Indexing DB", leave=False):
            arr = np.load(f)
            arr = smooth_pitch(arr) # Always smooth DB tracks

            # Windowing (3s windows, 1.5s hop)
            windows = []
            offsets = []
            i = 0
            while i + 300 <= len(arr):
                crop = arr[i:i+300]
                if np.mean(crop > 0) > 0.1: # Skip silence
                    windows.append(crop)
                    offsets.append(i/100.0)
                i += 150

            if not windows: continue

            # Batch Inference
            w_tensor = torch.from_numpy(np.stack(windows)).float().unsqueeze(1).to(DEVICE)
            # Batch processing to avoid OOM on huge files
            batch_size = 512
            for k in range(0, len(w_tensor), batch_size):
                batch = w_tensor[k:k+batch_size]
                embeddings = model.forward_one(batch)
                db_embeds.append(embeddings)

            song_name = os.path.basename(f).replace(".npy","")
            for t in offsets:
                metadata.append((song_name, t))

    if not db_embeds: return None, None
    return torch.cat(db_embeds), metadata

def run_evaluation(model, db_tensor, db_meta, pitch_dir):
    files = sorted(glob.glob(os.path.join(pitch_dir, "*.npy")))
    # Pick random subset of 300 songs for evaluation speed
    # (Checking all 3000+ takes too long, 300 is statistically significant)
    if len(files) > 300:
        random.shuffle(files)
        files = files[:300]

    modes = ['Clean', 'Soft', 'Hard', 'Ultra']
    results = {m: {'top1':0, 'top5':0, 'top10':0} for m in modes}
    total_clips = 0

    model.eval()

    for f in tqdm(files, desc="Eval Loop"):
        full_arr = np.load(f)
        if len(full_arr) < 500: continue
        target_name = os.path.basename(f).replace(".npy","")

        # Take 3 random starting points per song
        starts = np.random.randint(0, len(full_arr)-300, 3)

        for s in starts:
            base_clip = full_arr[s:s+300]
            base_clip = smooth_pitch(base_clip) # Always smooth query

            total_clips += 1

            for mode in modes:
                aug_clip = augment_clip(base_clip, mode=mode.lower())
                q_tensor = torch.from_numpy(aug_clip).float().unsqueeze(0).unsqueeze(0).to(DEVICE)

                with torch.no_grad():
                    q_emb = model.forward_one(q_tensor)

                # Geometric Scoring
                dists = torch.cdist(q_emb, db_tensor)
                vals, inds = torch.topk(dists, k=50, largest=False)

                votes = defaultdict(float)
                for k in range(50):
                    idx = inds[0, k].item()
                    dist = vals[0, k].item()
                    m_name, m_time = db_meta[idx]
                    score = 1.0 / (dist + 1e-4)
                    votes[m_name] += score

                ranked = sorted(votes.items(), key=lambda x: x[1], reverse=True)
                top_names = [x[0] for x in ranked]

                if not top_names: continue

                if top_names[0] == target_name: results[mode]['top1'] += 1
                if target_name in top_names[:5]: results[mode]['top5'] += 1
                if target_name in top_names[:10]: results[mode]['top10'] += 1

    return results, total_clips

# ==============================================================================
# MASTER LOOP (RETRAINED MODELS)
# ==============================================================================
# Note: These names match the save logic in the training script
experiments = [
    ("RETRAINED_DeepCNN_Full_Dual.pth", DeepCNN),
    ("RETRAINED_CRNN_Full_Dual.pth",    CRNN),
]

print(f"🚀 STARTING RETRAINED MODEL EVALUATION")
print(f"📂 Pitch Dir: {PITCH_DIR}")
print(f"📂 Model Dir: {MODEL_DIR}")

for model_name, ModelClass in experiments:
    model_path = os.path.join(MODEL_DIR, model_name)

    if not os.path.exists(model_path):
        print(f"\n⚠️ SKIPPING {model_name} (File not found)")
        continue

    print(f"\n" + "="*60)
    print(f"🧪 TESTING: {model_name}")
    print("="*60)

    # 1. Load
    model = ModelClass(embed_dim=128).to(DEVICE)
    try:
        ckpt = torch.load(model_path, map_location=DEVICE)
        if 'model' in ckpt: model.load_state_dict(ckpt['model'])
        else: model.load_state_dict(ckpt)
        print("   ✅ Weights Loaded Successfully")
    except Exception as e:
        print(f"❌ Failed to load: {e}")
        continue

    # 2. Build DB
    db_tensor, db_meta = build_db_for_model(model, PITCH_DIR)
    if db_tensor is None:
        print("❌ DB Build Failed (No embeddings generated)")
        continue

    # 3. Eval
    res, total = run_evaluation(model, db_tensor, db_meta, PITCH_DIR)

    # 4. Report
    print(f"\n📊 RESULTS for {model_name} (Total Trials: {total})")
    for mode in ['Clean', 'Soft', 'Hard', 'Ultra']:
        t1 = res[mode]['top1'] / total * 100
        t5 = res[mode]['top5'] / total * 100
        t10 = res[mode]['top10'] / total * 100
        print(f"   {mode:5} | Top-1: {t1:.1f}% | Top-5: {t5:.1f}% | Top-10: {t10:.1f}%")

print("\n✅ DONE.")

🚀 STARTING RETRAINED MODEL EVALUATION
📂 Pitch Dir: /content/expanded_pitch
📂 Model Dir: /content/drive/MyDrive/FIND_TUNE/pitch_based_model/retrained_models

🧪 TESTING: RETRAINED_DeepCNN_Full_Dual.pth
   ✅ Weights Loaded Successfully
   Building Database from 2944 files...


Eval Loop: 100%|██████████| 300/300 [00:47<00:00,  6.30it/s]



📊 RESULTS for RETRAINED_DeepCNN_Full_Dual.pth (Total Trials: 900)
   Clean | Top-1: 19.0% | Top-5: 38.3% | Top-10: 50.4%
   Soft  | Top-1: 15.9% | Top-5: 36.0% | Top-10: 49.4%
   Hard  | Top-1: 0.6% | Top-5: 1.6% | Top-10: 2.8%
   Ultra | Top-1: 0.1% | Top-5: 1.8% | Top-10: 2.7%

🧪 TESTING: RETRAINED_CRNN_Full_Dual.pth
   ✅ Weights Loaded Successfully
   Building Database from 2944 files...


Eval Loop: 100%|██████████| 300/300 [00:57<00:00,  5.22it/s]


📊 RESULTS for RETRAINED_CRNN_Full_Dual.pth (Total Trials: 900)
   Clean | Top-1: 11.9% | Top-5: 28.7% | Top-10: 37.7%
   Soft  | Top-1: 11.1% | Top-5: 27.9% | Top-10: 37.4%
   Hard  | Top-1: 0.2% | Top-5: 1.2% | Top-10: 2.9%
   Ultra | Top-1: 0.4% | Top-5: 1.3% | Top-10: 2.6%

✅ DONE.



📊 RESULTS for RETRAINED_DeepCNN_Full_Dual.pth (Total Trials: 900)
   Clean | Top-1: 19.0% | Top-5: 38.3% | Top-10: 50.4%
   Soft  | Top-1: 15.9% | Top-5: 36.0% | Top-10: 49.4%
   Hard  | Top-1: 0.6% | Top-5: 1.6% | Top-10: 2.8%
   Ultra | Top-1: 0.1% | Top-5: 1.8% | Top-10: 2.7%

============================================================
🧪 TESTING: RETRAINED_CRNN_Full_Dual.pth
============================================================
   ✅ Weights Loaded Successfully
   Building Database from 2944 files...
Eval Loop: 100%|██████████| 300/300 [00:57<00:00,  5.22it/s]
📊 RESULTS for RETRAINED_CRNN_Full_Dual.pth (Total Trials: 900)
   Clean | Top-1: 11.9% | Top-5: 28.7% | Top-10: 37.7%
   Soft  | Top-1: 11.1% | Top-5: 27.9% | Top-10: 37.4%
   Hard  | Top-1: 0.2% | Top-5: 1.2% | Top-10: 2.9%
   Ultra | Top-1: 0.4% | Top-5: 1.3% | Top-10: 2.6%